# Creation of STAC collection for GEDI02

This script describes the creation of GEDI02 stac collection, which is published on OpenLandMap STAC (https://stac.openlandmap.org/GEDI02/collection.json). 

In [ ]:
import os
import json
import rasterio
import urllib.request
import pystac

from datetime import datetime, timezone, timedelta
from shapely.geometry import Polygon, mapping
from tempfile import TemporaryDirectory
from shapely import from_wkb
import duckdb
import contextily as cx
from shapely import geometry
import geopandas as gpd
import rasterio
import numpy as np
from rasterio.transform import from_bounds
from rasterio.features import rasterize
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

from minio import Minio
import struct
from shapely.geometry import Point
import geopandas as gpd
import math


## Part 1: Retrieve files from s3 buckets

In [ ]:
access_key='your_S3_key'
secret_access_key='your_S3_secret_key'
ip='your_s3_ip'
s3_config = {
'access_key': access_key,
'secret_access_key': 'secret_access_key',
'host': ip}
client = Minio(s3_config['host'], s3_config['access_key'], s3_config['secret_access_key'], secure=False) 

In [ ]:
# List all objects in the bucket
objects=client.list_objects("global", recursive=True, prefix="glidar/gedi-ard/level2/l2v002.gedi_20190418_20230316_go_epsg.4326_v20240827")
# Print file names
files =[]
for obj in [i for i in objects if i.object_name.endswith('gedi_l2ab.parquet')]:    
    files.append([obj.object_name,obj.size])

In [ ]:
gedi_columns=['delta_time', 'beamname', 'shotnumber', 'latitude', 'longitude',
       'elev_lowestmode', 'rh100', 'rh99', 'rh98', 'rh97', 'rh95', 'rh75',
       'rh50', 'rh25', 'sensitivity', 'night_flag', 'rh100_a1', 'rh100_a2',
       'rh100_a3', 'rh100_a4', 'rh100_a5', 'rh100_a6', 'rh99_a1', 'rh99_a2',
       'rh99_a3', 'rh99_a4', 'rh99_a5', 'rh99_a6', 'rh98_a1', 'rh98_a2',
       'rh98_a3', 'rh98_a4', 'rh98_a5', 'rh98_a6', 'rh97_a1', 'rh97_a2',
       'rh97_a3', 'rh97_a4', 'rh97_a5', 'rh97_a6', 'rh95_a1', 'rh95_a2',
       'rh95_a3', 'rh95_a4', 'rh95_a5', 'rh95_a6', 'rh75_a1', 'rh75_a2',
       'rh75_a3', 'rh75_a4', 'rh75_a5', 'rh75_a6', 'rh50_a1', 'rh50_a2',
       'rh50_a3', 'rh50_a4', 'rh50_a5', 'rh50_a6', 'rh25_a1', 'rh25_a2',
       'rh25_a3', 'rh25_a4', 'rh25_a5', 'rh25_a6', 'sensitivity_a1',
       'sensitivity_a2', 'sensitivity_a3', 'sensitivity_a4', 'sensitivity_a5',
       'sensitivity_a6', 'elev_lowestmode_a1', 'elev_lowestmode_a2',
       'elev_lowestmode_a3', 'elev_lowestmode_a4', 'elev_lowestmode_a5',
       'elev_lowestmode_a6', 'cover', 'num_detectedmodes', 'omega', 'pai',
       'pgap_theta', 'rg', 'rv', 'rhog', 'selected_rg_algorithm', 'rhov',
       'selected_l2a_algorithm', 'fhd_normal', 'surface_flag', 'leaf_off_flag',
       'l2b_quality_flag', 'lon', 'lat', 'year']

## Part 2: define fucntions

1. transfer geoparquet wbk to wkt
2. rasterize goepandas dataframe to geotiff

In [ ]:
def transfer(geom_byte):
    # Given bytearray
    byte_data = geom_byte

    # Extract X and Y coordinates (assuming IEEE 754 double precision format)
    x, y = struct.unpack("dd", byte_data[-16:])  # Extract last 16 bytes as two doubles

    # Create a Shapely Point
    point = Point(x, y)
    return point


In [ ]:
def rasterize_gdf_to_geotiff(gdf, column, output_tiff, resolution=10, nodata_value=0):
    """
    Rasterizes a GeoDataFrame using values from a specific column and saves it as a GeoTIFF.
    
    Parameters:
    - gdf: GeoDataFrame containing geometries.
    - column: Column name whose values will be burned into the raster.
    - output_tiff: Output path for the GeoTIFF file.
    - resolution: Pixel resolution in the units of the CRS.
    """
    # Get bounds of the vector data
    minx, miny, maxx, maxy = gdf.buffer(resolution).total_bounds

    # Define raster width and height
    width = int((maxx - minx) / resolution)
    height = int((maxy - miny) / resolution)

    # Define transformation (maps pixel coordinates to spatial coordinates)
    transform = from_bounds(minx, miny, maxx, maxy, width, height)

    # Create a list of (geometry, value) tuples
    shapes = [(geom, value) for geom, value in zip(gdf.geometry, gdf[column])]

    # Rasterize vector data into an array
    raster = rasterize(
        shapes=shapes,
        out_shape=(height, width),
        transform=transform,
        fill=nodata_value,  # Background value
        all_touched=True,  # If True, touches any pixel it overlaps
    )
    
    # Normalize raster values to range [0, 1] for colormap
    min_val, max_val = raster[raster > nodata_value].min(), raster.max()
    normalized_raster = (raster - min_val) / (max_val - min_val)
    normalized_raster[raster == nodata_value] = 0  # Keep NoData as 0

    # Apply "magma" colormap from Matplotlib
    cmap = plt.cm.magma
    rgba_img = cmap(normalized_raster)  # Returns an (H, W, 4) array with RGBA values

    # Convert RGBA to 3-band (RGB) array (scale to 0–255)
    rgb_raster = (rgba_img[:, :, :3] * 255).astype(np.uint8)

    # Save to Cloud-Optimized GeoTIFF (COG)
    with rasterio.open(
        output_tiff, "w",
        driver="COG",  # Saves as a Cloud-Optimized GeoTIFF
        height=height, width=width,
        count=1, dtype=rasterio.uint8,  # 3 bands (RGB)
        crs=gdf.crs, transform=transform,
        nodata=0,  # Set NoData value
        compress="DEFLATE",  # Apply compression
        tiled=True,  # Enable tiling for COG
        blockxsize=256, blockysize=256,  # Set block size for COG
        BIGTIFF="IF_NEEDED"  # Allow large files
    ) as dst:
        dst.write(raster, 1)  # one band
        
        #dst.write(rgb_raster[:, :, 0], 1)  # Red band
        #dst.write(rgb_raster[:, :, 1], 2)  # Green band
        #dst.write(rgb_raster[:, :, 2], 3)  # Blue band
    
    print(f"Raster saved to {output_tiff}")

## Part 3: Process the files into STAC items and create metadata

STAC items of GEDI02 collection contains the infos: (1) url points to the single geoparquet file in S3 server, (2) an overview image of rasterized GEDI points as COG as thumbnail, (3) auxiliary metadata, such as footprint, bbox, platform, license.

In [ ]:
def worker(file):
    url='https://s3.opengeohub.org/global/'+file[0]
    asset_name = file[0].split('/')[3]
    file_size = file[1]

    df_duckdb = duckdb.sql(f"""
                            INSTALL httpfs;
                            LOAD httpfs;
                            INSTALL spatial;
                            LOAD spatial;

                            SELECT longitude, latitude, rh95, delta_time, geometry

                            FROM "{url}"
                            """)

    df=df_duckdb.df()
    df['geometry']=df.geometry.apply(lambda x: transfer(x))
    df['datetime'] = [datetime(2018,1,1,0,0,0) + timedelta(seconds=i) for i in df['delta_time']]
    num_of_points=len(df)

    s_date=min(df.datetime)
    e_date=max(df.datetime)

    item_name = '_'.join(file[0].split('/')[4:]).split('.')[0]


    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326"
    )
    bbox=gdf.total_bounds.tolist()
    xmin=math.floor(bbox[0])
    ymin=math.ceil(bbox[1])
    xmax=math.floor(bbox[2])
    ymax=math.ceil(bbox[3])

    footprint = mapping(geometry.box(xmin,ymin,xmax,ymax))
    if len(gdf)>5000:
        gdf = gdf.sample(5000)
    os.makedirs(f'stac/GEDI02/{item_name}',exist_ok=True)
    thumbmail_path = f'stac/GEDI02/{item_name}/overview_{item_name}.tif'
    rasterize_gdf_to_geotiff(gdf,'rh95',thumbmail_path,0.01, nodata_value=0)
    
    item = pystac.Item(id=item_name,
                     geometry=footprint,
                     bbox=gdf.total_bounds.tolist(),
                     properties={'size (bytes)':file_size,'point counts':num_of_points},
                     start_datetime=s_date,
                     end_datetime=e_date,
                     datetime=None,
                     stac_extensions='https://github.com/Open-Earth-Monitor/GlobalEarthPoint',                   
                     )

    item.common_metadata.platform = 'GlobalEarthPoint'
    item.common_metadata.license = 'CC-BY-4.0'
    # Define the COG asset with media type
    item.add_asset(
        key=asset_name,
        title='GeoParquet',
        roles=["data"],        
        asset=pystac.Asset(
            href=url
            description='A partition of high quality GEDI02 dataset in GeoParquet.'
        )

    )
    item.assets["thumbnail"] = pystac.Asset(
        href=f'overview_{item_name}.tif',
        media_type="image/tiff; application=geotiff; profile=cloud-optimized",
        roles=["overview","thumbnail"],
        title="Overview",
        description="A COG representing rh95 rasterized from GEDI02 points (vector data), 1km."
    )
    # Add Asset and all its information to Item 
    

    return item


In [ ]:
items = Parallel(n_jobs=-1)(delayed(worker)(i) for i in files)

# Part 4: create a geojson of tile system overview at the collection level.
The GeoJSON overview contains the tiling system (5x5 degree) yearly, the size, point counts, startdate and enddate of the record, etc. The purpose of this file is inspired by [stac-geoparquet](https://github.com/stac-utils/stac-geoparquet), where the overview of collection can help filter the collection without loading the items themselves. 

In [ ]:
df=[]
for item in items:
    d={'item_id':item.id,
'size_in_mb':item.properties['size (bytes)']/(1024**2),
'point_counts':item.properties['point counts'],
'start_date':item.properties['start_datetime'],
'end_date':item.properties['end_datetime'],
'platform':item.properties['platform'],
'stac_extentsion':item.stac_extensions,
'asset_file':item.assets['l2v002.gedi_20190418_20230316_go_epsg.4326_v20240827'].href,
'geometry':Polygon(item.geometry['coordinates'][0])}
    df.append(d)


In [ ]:
df = pd.DataFrame(df)

In [ ]:
gdf = gpd.GeoDataFrame(
    df, crs="EPSG:4326"
)

In [ ]:
gdf=gdf.set_geometry('geometry')

In [ ]:
gdf.to_file('stac/GEDI02/stac_items_geojson', driver='GeoJSON')

## Part 5: Create a collection placeholder 

Create a collection and define the spatial and temporal extent, license, columns, keywords, etc..

In [ ]:
collection_bbox = [-180, 51.6, 180 ,-51.6]
collection_interval = [datetime(2019,3,25), datetime(2023,3,15)]

In [ ]:
spatial_extent = pystac.SpatialExtent(bboxes=[collection_bbox])
temporal_extent = pystac.TemporalExtent(intervals=[collection_interval])

In [ ]:
collection_extent = pystac.Extent(spatial=spatial_extent, temporal=temporal_extent)

In [ ]:
collection = pystac.Collection(id='GEDI02',
                               title='High quality GEDI level2',
                               description="Global Ecosystem Dynamics Investigation (GEDI) is a full-waveform satellite LiDAR (Light Detection And Ranging), collecting 3D measurements over land (trough recorded backscattered laser energy, i.e., waveforms) near-globally, between 51.6° N and S within a laser footprint diameter of 25 m between 2019-03-25 and 2023-03-15. To foster the uptake and mask out the complexity and large data volume of GEDI products (GEDI L2, version 2,- V002) to the OEMC  and broader users, we aim to host pre-filtered and high-quality GEDI points (including their relative heights, and waveform- and derived-attributes) and offer them as a cloud service.",
                               extent=collection_extent,
                               license='CC-BY-4.0',
                               keywords=['gedi','level2','in-situ data','lidar','canopy height','canopy structure','terrain height'],
                               extra_fields={'columns_provided':gedi_columns})

In [ ]:
# Define a new provider
new_provider = pystac.Provider(
    name="OpenGeoHub",
    description="OpenGeoHub Foundation",
    roles=["processor", "host"],  # Can be 'producer', 'processor', 'host', 'licensor'
    url="http://opengeohub.org"  # Provider's website
)

# Add the provider to the collection (or update existing ones)
collection.providers = [new_provider]

In [ ]:
# Add DOI, Contact Name, and Email to extra_fields
collection.extra_fields["doi"] = "https://doi.org/10.5281/zenodo.8406375"  # Replace with actual DOI
collection.extra_fields["contact_name"] = "Yu Feng HO"
collection.extra_fields["contact_email"] = "yu-feng.ho@opengeohub.org"


## Part 6: Insert objects into Collection

In [ ]:
# insert overview of the collection (GeoJSON) 
tiles_vector = pystac.Asset(title='GeoJSON STAC items',href='stac_items_geojson', 
                          media_type=pystac.MediaType.GEOJSON,
                          description="STAC items of GEDI02 collection based on a 5 degree x 5 degree tiling system")
collection.add_asset(key='stac_items',
                   asset=tiles_vector)

In [ ]:
# insert items 
collection.add_items(items)

## Part 7: Create a self-contained catalog to host the collection

In [ ]:
catalog = pystac.Catalog(id='GlobalEarthPoint',
                         description='This Catalog serves the cloud optimized high equality vector data of the world')
catalog.add_child(collection)

In [ ]:
#catalog.normalize_hrefs(os.path.join('/mnt/apollo/eu_ecudatacube_vector', "stac"))
catalog.normalize_hrefs("stac")

In [ ]:
catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED,dest_href='stac')